# RSA Implementation and common mistakes

## How to set-up RSA for Dummies
The first problem we have is generating primes of sufficient size. Let's start by making a little function to check if big numbers are prime

In [258]:
from random import randint
smallPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 
            127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 
            257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 
            401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

def isPrime(n):
    # Quickly test small factors.
    if n in smallPrimes:
            return True

    # Use Fermat's little theorem to do the harder work on larger primes. (https://en.wikipedia.org/wiki/Fermat%27s_little_theorem)
    for i in range(0, 50): 
        base = randint(2, n-1)
        if pow(base, n-1, n) != 1: 
            return False
    
    return True

And let's check we've not been completely stupid by checking a few cases:

In [259]:
print(2, isPrime(2))
print(3, isPrime(3))
print(10, isPrime(10))

print(65537, isPrime(65537))
print(65539, isPrime(65539))
print(65541, isPrime(65541))



2 True
3 True
10 False
65537 True
65539 True
65541 False


And now for something a bit bigger. 

This prime was nicked from the [RFC 3526](https://www.rfc-editor.org/rfc/rfc3526.txt). 

In [260]:
bigPrime = """FFFFFFFF FFFFFFFF C90FDAA2 2168C234 C4C6628B 80DC1CD1
      29024E08 8A67CC74 020BBEA6 3B139B22 514A0879 8E3404DD
      EF9519B3 CD3A431B 302B0A6D F25F1437 4FE1356D 6D51C245
      E485B576 625E7EC6 F44C42E9 A637ED6B 0BFF5CB6 F406B7ED
      EE386BFB 5A899FA5 AE9F2411 7C4B1FE6 49286651 ECE45B3D
      C2007CB8 A163BF05 98DA4836 1C55D39A 69163FA8 FD24CF5F
      83655D23 DCA3AD96 1C62F356 208552BB 9ED52907 7096966D
      670C354E 4ABC9804 F1746C08 CA18217C 32905E46 2E36CE3B
      E39E772C 180E8603 9B2783A2 EC07A28F B5C55DF0 6F4C52C9
      DE2BCBF6 95581718 3995497C EA956AE5 15D22618 98FA0510
      15728E5A 8AACAA68 FFFFFFFF FFFFFFFF"""

bigPrime = bigPrime.replace(" ", "")
bigPrime = bigPrime.replace("\n", "")

bigDaddy = int(bigPrime, 16)

isPrime(bigDaddy)

True

Alright, so how do we generate a large random prime. The code below implements the process:

In [261]:
def bigPrime(bits):
    from random import getrandbits
    p = getrandbits(bits)
    while not isPrime(p):
        p = getrandbits(bits)
    return p

Does this prove that the numbers are prime? Of course not, they're sort of "industrial strength" primes.

Alrighty then, let's generate some primes:

In [262]:
p = bigPrime(1024)
q = bigPrime(1024)

p,q

(133130445491212145520478326441409709283291423178470109782975496713539201509380469129158860234024928826882744532790807463353370316261109075070843062154683744002614957378888117062512815658837648553451977261644723850230561353760118379138323522924090714973846458425942681178673040084048356418518872315165465220033,
 141908663200565974701123062962647743019751568166833687946556797248869128053850868048716225186195531236496258289977648085244861135135841973290976964902815015015773677077786011048263943946520703527471605675871987613817338949744818593175515657786361369119950301008744006515422023204843369472653676292776540723931)

And multiply to get a modulus:

In [263]:
n = p * q
n

18892363550953731382512216549102948606458647605997463372813292230034576272056310415115762784827605337383369710470016725449518590305209047528702959702753594647099550917680196353769570751738966481104372683631663107050635565434205818085230944348215843339760028317443154062727893567952475981103017836956517865508577226785610542515175731151514749845124381898102645633291012733448295478488543265555642911587236621243730754416148951969065349237970499909000193161073846998560269302326410931618134466152533857087157648354871491939820108934073344965875763770792424437380745240147549180951716006431767226466479909650828023709723

# Explanation of the role of Extended GCD / LCM

The extended Euclian algorithm is the standard GCD algorithm extended to show more of its internal state. It's used to find solutions to the equations of the form:

$$(ax + by) = gcd(x, y)$$

In our case, we're looking for a solution such that:

$$e \times d \equiv {1 \mod {\text{lcm} (p-1,q-1)}}$$

So, if we substitute:

$$ex + (b \times {\text{lcm} (p-1,q-1)}) = gcd(e, {\text{lcm} (p-1,q-1)})$$

But by requirement for the scheme to work:

$$gcd(e, {\text{lcm} (p-1,q-1)}) = 1$$

So we can rewrite the right hand side to 1:

$$(e \times x) + (b \times {\text{lcm} (p-1,q-1)}) = 1$$

Now we take both sides  $\mod {\text{lcm} (p-1,q-1)}$:

$$((e \times x) + (b \times {\text{lcm} (p-1,q-1)})) \mod {\text{lcm} (p-1,q-1)} \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

But notice that we just have $b$ multiplied by the thing in the mod, so we can simplify:

$$ex \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

This looks like our target equation above, so $x=d$ which is our encryption exponent.

In [264]:
def gcdExtended(a, b):
		if a == 0 :   
			return b,0,1
		     
		gcd,x1,y1 = gcdExtended(b % a, a)  
     
		x = y1 - (b//a) * x1  
		y = x1  
	     
		return gcd,x,y 

def lcm(a, b):
	gcd, _, _ = gcdExtended(a, b)
	return (a * b) // gcd


Right, now we get to the business end of setting up RSA. First we need to compute phi(n), which is the number of numbers relatively prime to "n"

In [265]:
phi_n = lcm(p-1, q-1)

Next we need to define what we want for the encryption exponent. It needs to be big enough so that when we do m^e it is bigger than p and thus "loops around"

In [239]:
e = 65537

Now the final step, let's compute d from phi_n and e.

In [266]:
_, d, _ = gcdExtended(e, phi_n)

if d < 0:
	d = phi_n + d # d is negative. Lost a good 45 minutes there.

d

119092785310176638862870349693800873283063396293750594180790183329118108764084099002625738617627541376956142421389889189820861884982387329934825688179671609608068699238957493453698842671744128642182279184952262579078693142320429637736059809734466817145058508028951128853268374340998884102398866258706421097545403368586323616064295391799535863009106841581653346712315613878048576704657093415703649575797850697604832376223046630471200153123375980304745679110272292334156901131647239526610050932937501786784239425352958793310656170058917519221374366819225889107972964394153920415874584225916611761722366815138772252033

Alright, let's check everything works!!!

In [267]:
m = 420

cipherText = pow(m, e, n)
plainText = pow(cipherText, d, n)

m, e, plainText

(420, 65537, 420)

# Now let's put the whole thing in to one neat package

In [268]:
class RSAFixedKey:
    def __init__(self, e=65537):
        self.p = bigPrime(1024)
        self.q = bigPrime(1024)
        self.n = self.p * self.q
        self.e = e
        phi_n = lcm((self.p-1),(self.q-1))
        _, self.d, _ = gcdExtended(e, phi_n)
        if self.d < 0:
            self.d = phi_n + self.d

    def encrypt(self, data):
        return pow(data, self.e, self.n)

    def decrypt(self, data):
        return pow(data, self.d, self.n)

And let's test it out!

In [269]:
rsa = RSAFixedKey()

plainTextBefore = 69
cipherText=rsa.encrypt(69)
plainTextAfter=rsa.decrypt(cipherText)

plainTextBefore, cipherText, plainTextAfter


(69,
 19154571854219123118017408917716215318225928995574100265451993774620852654933216924714111483015469116247834665962470277924220952689276325757573546374476457852238490238109605879693812706634809098345056594875433597333694912946011917981328968771058020603847165470643472802431995808263482466171068746417056851302819612183812968741112098029829055994177020315464685719045539330649373149513038253553018091964301164518857673813760061706705014244884983227779363041551049773986525706238355939455460171452203151292432218002004379792757575128955512600227534528976876690477328226118712130275458174146330886873137273582197869776168,
 69)

# Attack: Not picking a big enough exponent for $e$

In [270]:
rsa = RSAFixedKey(e=3)

In [271]:
from SemanticSecurityGame import SemanticSecurityGame

class RSAGameStrategyLowExponent:
    def getMessages(self, trialNumber):
           m0 = 3
           m1 = 2
           
           return m0, m1

    def challenge(self, challenge, trialNumber):
           result = challenge ** (1/3)

           if result == 3:
                return 0 
           else:
                return 1

strategy = RSAGameStrategyLowExponent()
game = SemanticSecurityGame(rsa, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 50}

# Attack: The cipher-text is the same each time!

In [272]:
rsa = RSAFixedKey()

In [274]:
from SemanticSecurityGame import SemanticSecurityGame

class RSAGameStrategy:
    def getMessages(self, trialNumber):
            if trialNumber == 0:
                m0 = 69
                m1 = 69
            else:
                m0 = 69
                m1 = 420
            return m0, m1 

    def challenge(self, challenge, trialNumber):
            if (trialNumber == 0):
                self.savedMessage = challenge
            if self.savedMessage == challenge:
                return 0
            else:
                return 1

strategy = RSAGameStrategy()
            
game = SemanticSecurityGame(rsa, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 50}